In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py

import kalepy as kale

import holodeck as holo
from holodeck.constants import MSOL, PC

In [ ]:
FNAME_ILLUSTRIS_DATA = (
    "/Users/lzkelley/Research/NANOGrav/holodeck/"
    "data_external/galaxy-mergers_L75n1820FP_gas-100_dm-100_star-100_bh-000.hdf5"
)

In [ ]:
def plot_bin_pop(bin_pop):
    # Plot binary parameters
    mt, mr = holo.utils.mtmr_from_m1m2(bin_pop.mass)
    redz = holo.utils.a_to_z(bin_pop.time)
    data = [mt/MSOL, mr, bin_pop.sepa/PC, 1+redz]
    data = [np.log10(dd) for dd in data]
    reflect = [None, [None, 0], None, [0, None]]
    labels = ['M', 'q', 'a', 'z']
    if bin_pop.eccs is not None:
        data.append(bin_pop.eccs)
        reflect.append([0.0, 1.0])
        labels.append('e')
    
    kde = kale.KDE(data, reflect=reflect)
    corner = kale.Corner(kde, labels=labels, figsize=[8, 8])
    corner.plot_data(kde)
    return corner

## Construct Illustris-Based Binary Population

In [ ]:
bin_pop = holo.holodeck.BP_Illustris(FNAME_ILLUSTRIS_DATA)
ill_name = os.path.basename(bin_pop._fname).split('_')[1]
print("Loaded", bin_pop.size, "binaries from Illustris", ill_name)

In [ ]:
plot_bin_pop(bin_pop)
plt.show()

## Apply a modifier to add (arbitrary) eccentricities, and resample 5x

In [ ]:
mod_ecc = holo.holodeck.PM_Eccentricity()
mod_resamp = holo.holodeck.PM_Resample(resample=5.0)
mods = [mod_ecc, mod_resamp]
for mod in mods:
    mod(bin_pop)
    
bin_pop._update_derived()
    
print("Population now has", bin_pop.size, "elements")

In [ ]:
plot_bin_pop(bin_pop)
plt.show()